## 2. Clasificación con SVM

Como se ha mencionado en el preprocesamiento, en el problema que aquí se estudia, cada individuo puede pertenecer a una clase, las dos o ninguna, por lo que se está tratando de un problema multietiqueta. Para usar una máquina de soporte vectorial en clasificación es necesario codificar la salida para que las distintas variantes de SVM pueda realizar la clasificación.

In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.svm import LinearSVC, SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler



In [13]:
X_train = pd.read_csv("preprocessed_training_dataset.csv")
y_train = pd.read_csv("training_set_labels.csv", index_col="respondent_id")

# Cargar el dataset de prueba
X_test = pd.read_csv("preprocessed_test_dataset.csv")

In [14]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape)


Train set: (26707, 48) (26707, 2)
Test set: (26708, 48)


In [24]:
X_test.head()

,h1n1_concern,h1n1_knowledge,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,...,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA,household_adults,household_children
0,2.0,2.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,3.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,1.0,4.0,1.0,1.0,4.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
2,2.0,2.0,5.0,4.0,2.0,5.0,4.0,4.0,2.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,4.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,3.0,1.0,5.0,2.0,4.0,4.0,4.0,2.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [25]:
X_train.head()

,h1n1_concern,h1n1_knowledge,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,...,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA,household_adults,household_children
0,1.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3.0,2.0,5.0,4.0,4.0,4.0,2.0,4.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,3.0,1.0,1.0,4.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0
3,1.0,1.0,3.0,3.0,5.0,5.0,4.0,1.0,4.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.0,1.0,3.0,3.0,2.0,3.0,1.0,4.0,3.0,2.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


Antes de proceder, es necesario escalar los valores, sobre todo para que las variables de mayor rango (pertenencientes a las preguntas de opinión mayoritariamente) no influyan en el cálculo de las distancias con SVM. Dentro de las posibilidades de escalado, se puede optar por:
- Normalización zero-mean: Escala los valores en un rango $[0,1]$ en función de la media y la varianza de la variable. Esta normalización se usa cuando las distribuciones de las variables siguen aproximadamente una distribución normal.
- Normalización MinMaxScaling: Escala los valores en un rango $[0,1]$ en función de los valores máximos y mínimos de cada variable. Esta técnica de escalado conserva las características de los datos originales tras la transformación. 

Dado que no se conocen las distribuciones originales de los datos, se va a optar por el uso de escalado MinMax

In [15]:
# Escalar características
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[0.66666667, 1.        , 1.        , ..., 0.        , 0.33333333,
        0.        ],
       [0.33333333, 0.5       , 0.75      , ..., 1.        , 1.        ,
        0.        ],
       [0.66666667, 1.        , 1.        , ..., 1.        , 0.33333333,
        0.        ],
       ...,
       [0.        , 0.5       , 0.75      , ..., 0.        , 0.33333333,
        0.        ],
       [1.        , 0.5       , 0.25      , ..., 0.        , 0.33333333,
        0.        ],
       [0.66666667, 0.5       , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
# Asegurarse de que todos los valores sean numéricos
#X_train = X_train.astype(float)
#X_test = X_test.astype(float)
y_train = y_train.astype(int)

In [17]:
from sklearn.model_selection import train_test_split

X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    X_train_scaled, y_train, test_size=0.33, random_state=42
)

Una vez tenemos todos los datos en el mismo formato, podemos pasar a implementar SVM. Este algoritmo trata de sobredimensionar los datos para poder generar un hiperplano de separación entre dos clases, por lo que, de manera nativa no es capaz de realizar una clasificación multietiqueta. Para solventar este problema se usa un pequeño 'truco' a la hora de implementar el algoritmo, que es la descomposición binaria de la respuesta obtenida, de esta manera SVM puede abordar los problemas multietiqueta como problemas independientes:

- Si se entrena un clasificador para cada par de valores posibles se está usando una estrategia OVO (One-vs-One)
- Si el clasificador es capaz de distinguir en cada caso su etiqueta frente a las demás se usa una estrategia OVR (One-vs-Rest)

Otra opción para la clasificación multietiqueta pasa por el uso de la función 'MultiOutputClassifier' implementada en 'sklearn', capaz de tratar cada etiqueta como un problema binario independiente, entrenando un clasificador para cada etiqueta sin comparar con las demás:


#### Clasificación SVM con RBF y balanceo de clases

##### Optimización por GridSearch y clases balanceadas

In [ ]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
import numpy as np


base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Paso 2: Configurar MultiOutputClassifier para manejar multietiqueta
multi_label_svm = MultiOutputClassifier(base_svm)

# Paso 3: Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10],       # Regularización
    'gamma': ['scale', 'auto'],   # Parámetro del kernel RBF
    'max_iter': [10000, 50000, 100000],
    'kernel': ['rbf', 'precomputed']
}

# Paso 4: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=multi_label_svm,
    param_grid=param_grid,
    cv=3,  # Validación cruzada de 5 pliegues
    scoring='roc_auc',  # Métrica para evaluar el modelo
    verbose=3,  # Para ver el progreso
    n_jobs=-1,   # Usar todos los núcleos disponibles
    refit=True

)

# Paso 5: Entrenar con los datos de entrenamiento
grid_search.fit(X_train_split, y_train_split)

# Paso 6: Resultados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_valid_split)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en optimización (train): {grid_search.best_score_}")
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))
roc_auc = roc_auc_score(y_valid_split, y_valid_pred, average='macro', multi_class='ovr')

# Imprimir el resultado
print(f"ROC AUC Score (test): {roc_auc:.4f}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Mejores parámetros encontrados:  {'estimator__C': 1, 'estimator__gamma': 'auto', 'estimator__kernel': 'rbf', 'estimator__max_iter': 10000}


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6356932153392331
ROC AUC Score en optimización (train): 0.8374914602749269
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.48      0.71      0.58      1868
           1       0.75      0.78      0.76      4044

   micro avg       0.64      0.76      0.69      5912
   macro avg       0.62      0.74      0.67      5912
weighted avg       0.66      0.76      0.70      5912
 samples avg       0.35      0.37      0.35      5912

ROC AUC Score (test): 0.7651


#### Optimización de parámetros con RandomSearch y clases balanceadas

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Configurar MultiOutputClassifier
multi_label_svm = MultiOutputClassifier(base_svm)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': np.logspace(-3, 1, 5),  # Ajuste más fino para 'C'
    'estimator__gamma': ["scale", "auto"],  # Combina números y strings válidos
    'estimator__max_iter': [10000, 50000, 100000],
    'estimator__kernel': ['rbf', 'precomputed']
}



# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_split, y_train_split)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Mejores parámetros: {'estimator__max_iter': 10000, 'estimator__gamma': 'auto', 'estimator__C': np.float64(1.0)}
Mejor score en train: 0.8370122504743737


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6356932153392331
ROC AUC Score en Test: 0.7651
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.48      0.71      0.58      1868
           1       0.75      0.78      0.76      4044

   micro avg       0.64      0.76      0.69      5912
   macro avg       0.62      0.74      0.67      5912
weighted avg       0.66      0.76      0.70      5912
 samples avg       0.35      0.37      0.35      5912



De aquí en adelante usaremos la optimización por Random search por ser más efectiva en el cálculo.

#### SVM RBF con clases Desbalanceadas

In [73]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)

# Configurar MultiOutputClassifier
multi_label_svm = MultiOutputClassifier(base_svm)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': np.logspace(-3, 1, 5),  # Ajuste más fino para 'C'
    'estimator__gamma': np.logspace(-3, 1, 5),  # Ajuste más fino para 'gamma'
    'estimator__max_iter': [10000, 50000],
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_split, y_train_split)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__max_iter': 50000, 'estimator__gamma': np.float64(0.01), 'estimator__C': np.float64(10.0)}
Mejor score en train: 0.8348147868911259
Accuracy en validación: 0.6655321080099841
ROC AUC Score en Test: 0.7203
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.69      0.38      0.49      1868
           1       0.77      0.73      0.75      4044

   micro avg       0.75      0.62      0.68      5912
   macro avg       0.73      0.55      0.62      5912
weighted avg       0.75      0.62      0.67      5912
 samples avg       0.33      0.31      0.31      5912



#### SVM Kernel Lineal y Balanceo de Clases

También podemos probar LinearSVC, en este caso el clasificador SVM que se usa kernel lineal y al no obtener probabilidades, no podemos calcular el roc_auc para evaluar el randomsearch:


Con balanceo de clases

In [80]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced'))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'estimator__max_iter': [10000, 20000, 50000],  # Opciones discretas
    'estimator__dual': [True, False]  # Opciones binarias
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_split, y_train_split)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.001, 'estimator__max_iter': 10000, 'estimator__dual': True, 'estimator__C': 0.1}
Accuracy en validación: 0.6371681415929203
ROC AUC Score en Test: 0.7648
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.49      0.71      0.58      1868
           1       0.76      0.76      0.76      4044

   micro avg       0.65      0.74      0.69      5912
   macro avg       0.62      0.73      0.67      5912
weighted avg       0.67      0.74      0.70      5912
 samples avg       0.34      0.37      0.34      5912



In [83]:
random_search.best_score_

np.float64(0.6556198980227645)

El mejor modelo,...................

#### SVM Kernel Lineal y Desbalanceo de Clases

In [81]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'estimator__max_iter': [10000, 20000, 50000],  # Opciones discretas
    'estimator__dual': [True, False]  # Opciones binarias
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_split, y_train_split)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_split, y_train_split)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_valid_split, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.01, 'estimator__max_iter': 20000, 'estimator__dual': True, 'estimator__C': 1}
Accuracy en validación: 0.6675743135920127
ROC AUC Score en Test: 0.7241
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.69      0.40      0.50      1868
           1       0.77      0.73      0.75      4044

   micro avg       0.75      0.63      0.68      5912
   macro avg       0.73      0.56      0.63      5912
weighted avg       0.74      0.63      0.67      5912
 samples avg       0.34      0.32      0.32      5912



In [82]:
random_search.best_score_

np.float64(0.6556198980227645)

#### One Vs One y One vs Rest

Comparamos los resultados con los que se obtienen a partir de OVO y OVR:

In [84]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

ovr = OneVsRestClassifier(LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, class_weight='balanced', tol= 0.001)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_valid_split, y_pred, zero_division=0))


Accuracy en validación: 0.6371681415929203
ROC AUC Score: 0.7648
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.49      0.71      0.58      1868
           1       0.76      0.76      0.76      4044

   micro avg       0.65      0.74      0.69      5912
   macro avg       0.62      0.73      0.67      5912
weighted avg       0.67      0.74      0.70      5912
 samples avg       0.34      0.37      0.34      5912



In [85]:
ovr = OneVsRestClassifier(SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced', C=10, gamma=0.01)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_valid_split, y_pred, zero_division=0))

c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6345586566825505
ROC AUC Score: 0.7631
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.48      0.70      0.57      1868
           1       0.75      0.77      0.76      4044

   micro avg       0.65      0.75      0.69      5912
   macro avg       0.62      0.73      0.67      5912
weighted avg       0.67      0.75      0.70      5912
 samples avg       0.35      0.37      0.35      5912



OVO

In [86]:
ovo = OneVsOneClassifier(LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, class_weight='balanced', tol= 0.001)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_valid_split, y_pred, zero_division=0))

c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6345586566825505
ROC AUC Score: 0.7631
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.48      0.70      0.57      1868
           1       0.75      0.77      0.76      4044

   micro avg       0.65      0.75      0.69      5912
   macro avg       0.62      0.73      0.67      5912
weighted avg       0.67      0.75      0.70      5912
 samples avg       0.35      0.37      0.35      5912



In [88]:
ovr =MultiOutputClassifier(OneVsOneClassifier(SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced', C=10, gamma=0.01))); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_split, y_train_split)
y_pred = ovr.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_valid_split, y_pred, zero_division=0))

c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6345586566825505
ROC AUC Score: 0.7631
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.48      0.70      0.57      1868
           1       0.75      0.77      0.76      4044

   micro avg       0.65      0.75      0.69      5912
   macro avg       0.62      0.73      0.67      5912
weighted avg       0.67      0.75      0.70      5912
 samples avg       0.35      0.37      0.35      5912



In [89]:
ovo =MultiOutputClassifier( OneVsOneClassifier(LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, class_weight='balanced', tol= 0.001))); # Usamos mismos parámetros que antes y la misma semilla
ovo.fit(X_train_split, y_train_split)
y_pred = ovo.predict(X_valid_split)
roc_auc = roc_auc_score(y_valid_split, y_pred)

print("Accuracy en validación:", accuracy_score(y_valid_split, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_valid_split, y_pred, zero_division=0))

Accuracy en validación: 0.6371681415929203
ROC AUC Score: 0.7648
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.49      0.71      0.58      1868
           1       0.76      0.76      0.76      4044

   micro avg       0.65      0.74      0.69      5912
   macro avg       0.62      0.73      0.67      5912
weighted avg       0.67      0.74      0.70      5912
 samples avg       0.34      0.37      0.34      5912



Parece claro que el método OVR con LinearSVC se acerca en mayor medida a nuestros resultados con MultiOutputClassifier. 

#### Simplificación del Problema

Como hemos mencionado anteriormente, SVM no trabaja directamente con problemas multietiqueta, por lo que se envuelve con MultiOutputClassifier. Otra opción válida es dividir inicialmente las etiquetas para descomponerlo en dos SVM distintos:

In [91]:
y=y_train["seasonal_vaccine"]
z=y_train["h1n1_vaccine"]

respondent_id
24706    0
5393     0
20898    1
3429     0
8731     0
        ..
21575    0
5390     0
860      0
15795    0
23654    0
Name: h1n1_vaccine, Length: 21365, dtype: int64

#### Vacuna estacional

In [101]:
x_train,x_test,y_train,y_test=train_test_split(X_train_scaled,y,random_state=42,test_size=0.2)

In [66]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Mejores parámetros: {'max_iter': 10000, 'gamma': np.float64(0.01), 'C': np.float64(10.0)}
Mejor score en train: 0.8463056572303893


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.7744290527892175
ROC AUC Score en Test: 0.7762
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.76      0.78      2891
           1       0.73      0.80      0.76      2451

    accuracy                           0.77      5342
   macro avg       0.77      0.78      0.77      5342
weighted avg       0.78      0.77      0.77      5342



##### Sin balanceo

In [93]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Mejores parámetros: {'max_iter': 10000, 'gamma': np.float64(0.01), 'C': np.float64(10.0)}
Mejor score en train: 0.8461012794615804


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.7796705353800075
ROC AUC Score en Test: 0.7766
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.79      0.81      0.80      2891
           1       0.77      0.74      0.75      2451

    accuracy                           0.78      5342
   macro avg       0.78      0.78      0.78      5342
weighted avg       0.78      0.78      0.78      5342



#### LinearSVM con balanceo

In [103]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced')

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.01, 'max_iter': 20000, 'dual': True, 'C': 1}
Mejor score en train: 0.8428105823411389
Accuracy en validación: 0.775926619243729
ROC AUC Score en Test: 0.7747
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.80      0.79      0.79      2891
           1       0.75      0.76      0.76      2451

    accuracy                           0.78      5342
   macro avg       0.77      0.77      0.77      5342
weighted avg       0.78      0.78      0.78      5342



#### LinearSVM Sin balanceo

In [104]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.01, 'max_iter': 10000, 'dual': True, 'C': 10}
Mejor score en train: 0.8428026594336577
Accuracy en validación: 0.7755522276301011
ROC AUC Score en Test: 0.7723
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.78      0.81      0.80      2891
           1       0.77      0.73      0.75      2451

    accuracy                           0.78      5342
   macro avg       0.77      0.77      0.77      5342
weighted avg       0.78      0.78      0.78      5342



#### Vacuna H1N1. Clases balanceadas

In [94]:
x_train,x_test,y_train,y_test=train_test_split(X_train_scaled,z,random_state=42,test_size=0.2)

In [95]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'max_iter': 50000, 'gamma': np.float64(0.01), 'C': np.float64(10.0)}
Mejor score en train: 0.829500864916958
Accuracy en validación: 0.7832272557094722
ROC AUC Score en Test: 0.7512
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.91      0.81      0.85      4212
           1       0.49      0.70      0.58      1130

    accuracy                           0.78      5342
   macro avg       0.70      0.75      0.72      5342
weighted avg       0.82      0.78      0.80      5342



#### Sin balanceo

In [96]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'max_iter': 50000, 'gamma': np.float64(0.01), 'C': np.float64(1.0)}
Mejor score en train: 0.8239213951879351
Accuracy en validación: 0.8335829277424186
ROC AUC Score en Test: 0.6746
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90      4212
           1       0.68      0.40      0.50      1130

    accuracy                           0.83      5342
   macro avg       0.77      0.67      0.70      5342
weighted avg       0.82      0.83      0.82      5342



#### LinearSVC balanceado

In [99]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced')

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.001, 'max_iter': 20000, 'dual': True, 'C': 1}
Mejor score en train: 0.8277192999254049
Accuracy en validación: 0.7806065144140771
ROC AUC Score en Test: 0.7527
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.91      0.80      0.85      4212
           1       0.49      0.70      0.58      1130

    accuracy                           0.78      5342
   macro avg       0.70      0.75      0.71      5342
weighted avg       0.82      0.78      0.79      5342



#### Sin Balanceo

In [100]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.001, 'max_iter': 50000, 'dual': True, 'C': 10}
Mejor score en train: 0.8271237574013132
Accuracy en validación: 0.8367652564582553
ROC AUC Score en Test: 0.6770
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      0.95      0.90      4212
           1       0.70      0.40      0.51      1130

    accuracy                           0.84      5342
   macro avg       0.78      0.68      0.71      5342
weighted avg       0.82      0.84      0.82      5342



# Ensembles


### Para LinearSVM con Balanceo entre clases

In [107]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.multioutput import MultiOutputClassifier


# 1. Modelo base SVM con MultiOutputClassifier para multietiqueta
base_svm = LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, tol=0.001, class_weight='balanced')

# 2. Bagging clásico con SVM
bagging_svm = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=0.8,   # Usa el 80% de las muestras
    max_features=1.0,  # Usa el 100% de las características
    bootstrap=True,
    random_state=42
)
# Envolvemos el bagging con MultiOutputClassifier
bagging_svm_multi = MultiOutputClassifier(bagging_svm)


# 3. Random Subspaces (Bagging con muestreo de características)
random_subspaces = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=1.0,   # Usa el 100% de las muestras
    max_features=0.5,  # Usa el 50% de las características
    bootstrap=False,
    random_state=42
)
# Envolvemos el Random Subspaces con MultiOutputClassifier
random_subspaces_multi = MultiOutputClassifier(random_subspaces)

# Lista de modelos para comparar
models = {
    "Bagging_SVM": bagging_svm_multi,
    "Random_Subspaces": random_subspaces_multi
}

# Entrenamiento y evaluación
results = {}
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train_split, y_train_split)

    # Predicción
    y_pred = model.predict(X_valid_split)

    # Métricas
    accuracy = accuracy_score(y_valid_split, y_pred)
    report = classification_report(y_valid_split, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_valid_split, y_pred)

    # Guardar resultados
    results[name] = {"accuracy": accuracy, "report": report}
    print(f"=== {name} ===")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)

=== Bagging_SVM ===
Accuracy: 0.6671204901293397
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.40      0.51      1868
           1       0.77      0.73      0.75      4044

   micro avg       0.75      0.63      0.68      5912
   macro avg       0.73      0.57      0.63      5912
weighted avg       0.74      0.63      0.67      5912
 samples avg       0.34      0.32      0.32      5912

=== Random_Subspaces ===
Accuracy: 0.6277513047424552
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.13      0.22      1868
           1       0.77      0.69      0.73      4044

   micro avg       0.78      0.51      0.62      5912
   macro avg       0.79      0.41      0.47      5912
weighted avg       0.79      0.51      0.57      5912
 samples avg       0.31      0.26      0.28      5912



#### Ensemble de SVM con kernel RBF

In [111]:

# 1. Modelo base SVM con MultiOutputClassifier para multietiqueta
base_svm = SVC(kernel='rbf', C=10, random_state=42,  max_iter=10000, gamma=0.01, class_weight='balanced')

# 2. Bagging clásico con SVM
bagging_svm = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=0.8,   # Usa el 80% de las muestras
    max_features=1.0,  # Usa el 100% de las características
    bootstrap=True,
    random_state=42
)
# Envolvemos el bagging con MultiOutputClassifier
bagging_svm_multi = MultiOutputClassifier(bagging_svm)


# 3. Random Subspaces (Bagging con muestreo de características)
random_subspaces = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=1.0,   # Usa el 100% de las muestras
    max_features=0.5,  # Usa el 50% de las características
    bootstrap=False,
    random_state=42
)
# Envolvemos el Random Subspaces con MultiOutputClassifier
random_subspaces_multi = MultiOutputClassifier(random_subspaces)

# Lista de modelos para comparar
models = {
    "Bagging_SVM": bagging_svm_multi,
    "Random_Subspaces": random_subspaces_multi
}

# Entrenamiento y evaluación
results = {}
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train_split, y_train_split)

    # Predicción
    y_pred = model.predict(X_valid_split)

    # Métricas
    accuracy = accuracy_score(y_valid_split, y_pred)
    report = classification_report(y_valid_split, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_valid_split, y_pred)

    # Guardar resultados
    results[name] = {"accuracy": accuracy, "roc_auc":roc_auc , "report": report}
    print(f"=== {name} ===")
    print(f"Accuracy: {accuracy}")
    print(f"Roc-AUC: {roc_auc}")
    print("Classification Report:")
    print(report)

c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base

=== Bagging_SVM ===
Accuracy: 0.6418198320853188
Roc-AUC: 0.7642296571868485
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.70      0.58      1868
           1       0.76      0.75      0.76      4044

   micro avg       0.66      0.74      0.69      5912
   macro avg       0.63      0.72      0.67      5912
weighted avg       0.68      0.74      0.70      5912
 samples avg       0.34      0.36      0.34      5912



c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base

=== Random_Subspaces ===
Accuracy: 0.47674154753800774
Roc-AUC: 0.7226042099584467
Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.88      0.47      1868
           1       0.70      0.81      0.75      4044

   micro avg       0.50      0.83      0.62      5912
   macro avg       0.51      0.84      0.61      5912
weighted avg       0.58      0.83      0.66      5912
 samples avg       0.33      0.41      0.35      5912

